<a href="https://colab.research.google.com/github/dandanloveJM/COVID-19/blob/main/COVID19_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
USE_SUMMARY = True
FIND_PDFS = True

In [ ]:
import os
#%%capture
!curl -O https://download.java.net/java/GA/jdk11/9/GPL/openjdk-11.0.2_linux-x64_bin.tar.gz
!mv openjdk-11.0.2_linux-x64_bin.tar.gz /usr/lib/jvm/; cd /usr/lib/jvm/; tar -zxvf openjdk-11.0.2_linux-x64_bin.tar.gz
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-11.0.2/bin/java 1
!update-alternatives --set java /usr/lib/jvm/jdk-11.0.2/bin/java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-11.0.2"

In [ ]:
#%%capture
!pip install pyserini==0.8.1.0
from pyserini.search import pysearch

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget "https://bit.ly/2yh2RiW" -O - | tar -xzvf - -C "/content/drive/My Drive/covid/"

In [7]:
import tensorflow as tf
import tensorflow_hub as hub

In [8]:


# Download the module, and uncompress it to the destination folder. 
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC "/content/drive/My Drive/covid/kaggle"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0./
./tfhub_module.pb
./variables/
./variables/variables.data-00000-of-00001
 97  745M   97  729M    0     0  34.0M      0  0:00:21  0:00:21 --:--:-- 36.4M./variables/variables.index
./assets/
./saved_model.pb
100  745M  100  745M    0     0  34.1M      0  0:00:21  0:00:21 --:--:-- 36.7M


In [9]:
!pip install transformers

     |████████████████████████████████| 1.1MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 24.4MB/s 
     |████████████████████████████████| 3.0MB 41.2MB/s 
     |████████████████████████████████| 890kB 51.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3dae640016c8ac3f694afd1072c23d8fe2ff4b01a16fce9a322a1b63ecb241b6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
cd /content/drive/My Drive/covid

/content/drive/My Drive/covid


In [11]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [12]:
cd transformers

/content/drive/My Drive/covid/transformers


In [ ]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

QA_MODEL = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_TOKENIZER = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_MODEL.to(torch_device)
QA_MODEL.eval()


In [14]:

if USE_SUMMARY:
    SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    SUMMARY_MODEL.to(torch_device)
    SUMMARY_MODEL.eval()

In [15]:
!pip install biopython
from Bio import Entrez, Medline

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import re

     |████████████████████████████████| 2.3MB 4.8MB/s 


In [16]:
query = 'Which non-pharmaceutical interventions limit tramsission'
keywords = '2019-nCoV, SARS-CoV-2, COVID-19, non-pharmaceutical interventions, npi'

In [17]:
luceneDir = "/content/drive/My Drive/covid/lucene-index-covid-2020-04-10"
print(luceneDir)

/content/drive/My Drive/covid/lucene-index-covid-2020-04-10


In [18]:
import json
searcher = pysearch.SimpleSearcher(luceneDir)
hits = searcher.search(query + '. ' + keywords)
n_hits = len(hits)
## collect the relevant data in a hit dictionary
hit_dictionary = {}
for i in range(0, n_hits):
    doc_json = json.loads(hits[i].raw)
    idx = str(hits[i].docid)
    hit_dictionary[idx] = doc_json
    hit_dictionary[idx]['title'] = hits[i].lucene_document.get("title")
    hit_dictionary[idx]['authors'] = hits[i].lucene_document.get("authors")
    hit_dictionary[idx]['doi'] = hits[i].lucene_document.get("doi")

## scrub the abstracts in prep for BERT-SQuAD
for idx,v in hit_dictionary.items():
    abs_dirty = v['abstract']
    # looks like the abstract value can be an empty list
    v['abstract_paragraphs'] = []
    v['abstract_full'] = ''

    if abs_dirty:
        # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
        # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
        # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


        if isinstance(abs_dirty, list):
            for p in abs_dirty:
                v['abstract_paragraphs'].append(p['text'])
                v['abstract_full'] += p['text'] + ' \n\n'

        # looks like in some cases the abstract can be straight up text so we can actually leave that alone
        if isinstance(abs_dirty, str):
            v['abstract_paragraphs'].append(abs_dirty)
            v['abstract_full'] += abs_dirty + ' \n\n'


Lets try doing a simple BERT-SQuAD QA model first and see how it does

In [19]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.compat.v1.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.compat.v1.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT('/content/drive/My Drive/covid/kaggle')

INFO:absl:resolver HttpCompressedFileResolver does not support the provided handle.
INFO:absl:resolver GcsCompressedFileResolver does not support the provided handle.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [20]:
import numpy as np
def reconstructText(tokens, start=0, stop=-1):
    tokens = tokens[start: stop]
    if '[SEP]' in tokens:
        sepind = tokens.index('[SEP]')
        tokens = tokens[sepind+1:]
    txt = ' '.join(tokens)
    txt = txt.replace(' ##', '')
    txt = txt.replace('##', '')
    txt = txt.strip()
    txt = " ".join(txt.split())
    txt = txt.replace(' .', '.')
    txt = txt.replace('( ', '(')
    txt = txt.replace(' )', ')')
    txt = txt.replace(' - ', '-')
    txt_list = txt.split(' , ')
    txt = ''
    nTxtL = len(txt_list)
    if nTxtL == 1:
        return txt_list[0]
    newList =[]
    for i,t in enumerate(txt_list):
        if i < nTxtL -1:
            if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                newList += [t,',']
            else:
                newList += [t, ', ']
        else:
            newList += [t]
    return ''.join(newList)


def makeBERTSQuADPrediction(document, question):
    ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
    ## 50 word overlaps on either end so that it can understand and check longer abstracts
    nWords = len(document.split())
    input_ids_all = QA_TOKENIZER.encode(question, document)
    tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
    overlapFac = 1.1
    if len(input_ids_all)*overlapFac > 2048:
        nSearchWords = int(np.ceil(nWords/5))
        quarter = int(np.ceil(nWords/4))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1536:
        nSearchWords = int(np.ceil(nWords/4))
        third = int(np.ceil(nWords/3))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1024:
        nSearchWords = int(np.ceil(nWords/3))
        middle = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    elif len(input_ids_all)*overlapFac > 512:
        nSearchWords = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    else:
        input_ids = [input_ids_all]
    absTooLong = False    
    
    answers = []
    cons = []
    for iptIds in input_ids:
        tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
        sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(iptIds) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b
        assert len(segment_ids) == len(iptIds)
        n_ids = len(segment_ids)
        #print(n_ids)

        if n_ids < 512:
            start_scores, end_scores = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids]).to(torch_device))
        else:
            #this cuts off the text if its more than 512 words so it fits in model space
            #need run multiple inferences for longer text. add to the todo
            print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
            absTooLong = True
            start_scores, end_scores = QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
        start_scores = start_scores[:,1:-1]
        end_scores = end_scores[:,1:-1]
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        #print(answer_start, answer_end)
        answer = reconstructText(tokens, answer_start, answer_end+2)
    
        if answer.startswith('. ') or answer.startswith(', '):
            answer = answer[2:]
            
        c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
        answers.append(answer)
        cons.append(c)
    
    maxC = max(cons)
    iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
    confidence = cons[iMaxC]
    answer = answers[iMaxC]
    
    sep_index = tokens_all.index('[SEP]')
    full_txt_tokens = tokens_all[sep_index+1:]
    
    abs_returned = reconstructText(full_txt_tokens)

    ans={}
    ans['answer'] = answer
    #print(answer)
    if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
        ans['confidence'] = -1000000
    else:
        #confidence = torch.max(start_scores) + torch.max(end_scores)
        #confidence = np.log(confidence.item())
        ans['confidence'] = confidence
    #ans['start'] = answer_start.item()
    #ans['end'] = answer_end.item()
    ans['abstract_bert'] = abs_returned
    ans['abs_too_long'] = absTooLong
    return ans

Now we can write a function to do an Open Domain QA on all the abstracts using BERT-SQUAD

In [21]:
from tqdm import tqdm
def searchAbstracts(hit_dictionary, question):
    abstractResults = {}
    for k,v in tqdm(hit_dictionary.items()):
        abstract = v['abstract_full']
        if abstract:
            ans = makeBERTSQuADPrediction(abstract, question)
            if ans['answer']:
                confidence = ans['confidence']
                abstractResults[confidence]={}
                abstractResults[confidence]['answer'] = ans['answer']
                #abstractResults[confidence]['start'] = ans['start']
                #abstractResults[confidence]['end'] = ans['end']
                abstractResults[confidence]['abstract_bert'] = ans['abstract_bert']
                abstractResults[confidence]['idx'] = k
                abstractResults[confidence]['abs_too_long'] = ans['abs_too_long']
                
    cList = list(abstractResults.keys())

    if cList:
        maxScore = max(cList)
        total = 0.0
        exp_scores = []
        for c in cList:
            s = np.exp(c-maxScore)
            exp_scores.append(s)
        total = sum(exp_scores)
        for i,c in enumerate(cList):
            abstractResults[exp_scores[i]/total] = abstractResults.pop(c)
    return abstractResults

In [22]:
answers = searchAbstracts(hit_dictionary, query)

100%|██████████| 10/10 [00:01<00:00,  8.37it/s]


In [23]:
def getrecord(id, db):
    handle = Entrez.efetch(db=db, id=id, rettype='Medline', retmode='text')
    rec = handle.read()
    handle.close()
    return rec

def pubMedSearch(terms, db='pubmed', mindate='2019/12/01'):
    handle = Entrez.esearch(db = db, term = terms, retmax=10, mindate=mindate)
    record = Entrez.read(handle)
    record_db = {}
    for id in record['IdList']:
        try:
            record = getrecord(id,db)
            recfile = StringIO(record)
            rec = Medline.read(recfile)
            if 'AB' in rec and 'AU' in rec and 'LID' in rec and 'TI' in rec:
                if '10.' in rec['LID'] and ' [doi]' in rec['LID']:
                    record_db['pm_'+id] = {}
                    record_db['pm_'+id]['authors'] = ' '.join(rec['AU'])
                    record_db['pm_'+id]['doi'] = '10.'+rec['LID'].split('10.')[1].split(' [doi]')[0]
                    record_db['pm_'+id]['abstract'] = rec['AB']
                    record_db['pm_'+id]['title'] = rec['TI']
        except:
            print("Problem trying to retrieve: " + str(id))
        
    return record_db


Entrez.email = 'pubmedkaggle@gmail.com'


In [24]:
def searchDatabase(question, keywords, pysearch, lucene_database, pm_kw = '', minDate='2019/12/01', k=10):
    ## search the lucene database with a combination of the question and the keywords
    searcher = pysearch.SimpleSearcher(lucene_database)
    hits = searcher.search(question + '. ' + keywords, k=k)
    n_hits = len(hits)
    ## collect the relevant data in a hit dictionary
    hit_dictionary = {}
    for i in range(0, n_hits):
        doc_json = json.loads(hits[i].raw)
        idx = str(hits[i].docid)
        hit_dictionary[idx] = doc_json
        hit_dictionary[idx]['title'] = hits[i].lucene_document.get("title")
        hit_dictionary[idx]['authors'] = hits[i].lucene_document.get("authors")
        hit_dictionary[idx]['doi'] = hits[i].lucene_document.get("doi")
        
    
   
    
    ## scrub the abstracts in prep for BERT-SQuAD
    for idx,v in hit_dictionary.items():
        abs_dirty = v['abstract']
        # looks like the abstract value can be an empty list
        v['abstract_paragraphs'] = []
        v['abstract_full'] = ''

        if abs_dirty:
            # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
            # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
            # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


            if isinstance(abs_dirty, list):
                for p in abs_dirty:
                    v['abstract_paragraphs'].append(p['text'])
                    v['abstract_full'] += p['text'] + ' \n\n'

            # looks like in some cases the abstract can be straight up text so we can actually leave that alone
            if isinstance(abs_dirty, str):
                v['abstract_paragraphs'].append(abs_dirty)
                v['abstract_full'] += abs_dirty + ' \n\n'
    ## Search collected abstracts with BERT-SQuAD
    answers = searchAbstracts(hit_dictionary, question)
    
    ## display results in a nice format
    displayResults(hit_dictionary, answers, question)

    ## save results to json


In [25]:
workingPath = '/content/drive/My Drive/covid'
import pandas as pd

from IPython.core.display import display, HTML

#from summarizer import Summarizer
#summarizerModel = Summarizer()
def displayResults(hit_dictionary, answers, question):
    
    question_HTML = '<div style="font-family: Times New Roman; font-size: 28px; padding-bottom:28px"><b>Query</b>: '+question+'</div>'
    #all_HTML_txt = question_HTML
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    

    for c in confidence:
        if c>0 and c <= 1 and len(answers[c]['answer']) != 0:
            if 'idx' not in  answers[c]:
                continue
            rowData = []
            idx = answers[c]['idx']
            title = hit_dictionary[idx]['title']
            authors = hit_dictionary[idx]['authors'] + ' et al.'
            doi = '<a href="https://doi.org/'+hit_dictionary[idx]['doi']+'" target="_blank">' + title +'</a>'

            
            full_abs = answers[c]['abstract_bert']
            bert_ans = answers[c]['answer']
            
            
            split_abs = full_abs.split(bert_ans)
            sentance_beginning = split_abs[0][split_abs[0].rfind('.')+1:]
            if len(split_abs) == 1:
                sentance_end_pos = len(full_abs)
                sentance_end =''
            else:
                sentance_end_pos = split_abs[1].find('. ')+1
                if sentance_end_pos == 0:
                    sentance_end = split_abs[1]
                else:
                    sentance_end = split_abs[1][:sentance_end_pos]
                
            #sentance_full = sentance_beginning + bert_ans+ sentance_end
            answers[c]['full_answer'] = sentance_beginning+bert_ans+sentance_end
            answers[c]['sentence_beginning'] = sentance_beginning
            answers[c]['sentence_end'] = sentance_end
            answers[c]['title'] = title
            answers[c]['doi'] = doi
            if 'pdfLink' in hit_dictionary[idx]:
                answers[c]['pdfLink'] = hit_dictionary[idx]['pdfLink']
                
        else:
            answers.pop(c)
    
    
    ## now rerank based on semantic similarity of the answers to the question
    cList = list(answers.keys())
    allAnswers = [answers[c]['full_answer'] for c in cList]
    
    messages = [question]+allAnswers
    
    encoding_matrix = embed_fn(messages)
    similarity_matrix = np.inner(encoding_matrix, encoding_matrix)
    rankings = similarity_matrix[1:,0]
    
    for i,c in enumerate(cList):
        answers[rankings[i]] = answers.pop(c)

    ## now form pandas dv
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    pandasData = []
    ranked_aswers = []
    for c in confidence:
        rowData=[]
        title = answers[c]['title']
        doi = answers[c]['doi']
        idx = answers[c]['idx']
        rowData += [idx]            
        sentance_html = '<div>' +answers[c]['sentence_beginning'] + " <font color='red'>"+answers[c]['answer']+"</font> "+answers[c]['sentence_end']+'</div>'
        
        rowData += [sentance_html, c, doi]
        pandasData.append(rowData)
        ranked_aswers.append(' '.join([answers[c]['full_answer']]))
    
    if FIND_PDFS or SEARCH_MEDRXIV:
        pdata2 = []
        for rowData in pandasData:
            rd = rowData
            idx = rowData[0]
            if 'pdfLink' in answers[rowData[2]]:
                rd += ['<a href="'+answers[rowData[2]]['pdfLink']+'" target="_blank">PDF Link</a>']
            elif FIND_PDFS:
                if str(idx).startswith('pm_'):
                    pmid = idx[3:]
                else:
                    try:
                        test = UrlReverse('https://doi.org/'+hit_dictionary[idx]['doi'])
                        if test is not None:
                            pmid = test.pmid
                        else:
                            pmid = None
                    except:
                        pmid = None
                pdfLink = None
                if pmid is not None:
                    try:
                        pdfLink = FindIt(str(pmid))
                    except:
                        pdfLink = None
                if pdfLink is not None:
                    pdfLink = pdfLink.url

                if pdfLink is None:

                    rd += ['Not Available']
                else:
                    rd += ['<a href="'+pdfLink+'" target="_blank">PDF Link</a>']
            else:
                rd += ['Not Available']
            pdata2.append(rowData)
    else:
        pdata2 = pandasData
        
    
    display(HTML(question_HTML))
    
    if USE_SUMMARY:
        ## try generating an exacutive summary with extractive summarizer
        allAnswersTxt = ' '.join(ranked_aswers[:6]).replace('\n','')
    #    exec_sum = summarizerModel(allAnswersTxt, min_length=1, max_length=500)    
     #   execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>BERT Extractive Summary:</b>: '+exec_sum+'</div>'

        answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
        summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                               num_beams=10,
                                               length_penalty=1.2,
                                               max_length=1024,
                                               min_length=64,
                                               no_repeat_ngram_size=4)

        exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
        execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; margin-bottom:1pt"><b>BART Abstractive Summary:</b>: '+exec_sum+'</div>'
        display(HTML(execSum_HTML))
        warning_HTML = '<div style="font-family: Times New Roman; font-size: 12px; padding-bottom:12px; color:#CCCC00; margin-top:1pt"> Warning this is an autogenerated summary based on semantic search of abstracts, always examine the sources before accepting this conclusion.  If the evidence only mentions topic in passing or the evidence is not clear, the summary will likely not clearly answer the question.</div>'
        display(HTML(warning_HTML))

#    display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>Body of Evidence:</b></div>'))
    
    if FIND_PDFS or SEARCH_MEDRXIV:
        df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link','PDF Link'])
    else:
        df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
        
    display(HTML(df.to_html(render_links=True, escape=False)))
    
#displayResults(hit_dictionary, answers, query)

In [26]:
displayResults(hit_dictionary, answers, query)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,kqntu6ej,"in impact of non-pharmaceutical interventions (npis) to reduce covid-19 mortality and healthcare demand , imperial college covid-19 response team, london, 2020.",0.514955,"Commentary on Ferguson, et al., “Impact of Non-pharmaceutical Interventions (NPIs) to Reduce COVID-19 Mortality and Healthcare Demand”",Not Available
1,vy1mu04l,"much of the uncertainty about the progression of the covid-19 pandemic stems from questions about when and how non-pharmaceutical interventions (npi) by governments, in particular social distancing measures , are implemented, to what extent the population complies with these measures, and how compliance changes through time.",0.436335,How will this continue? Modelling interactions between the COVID-19 pandemic and policy responses,Not Available
2,swcq2wfl,public health resorted to non-pharmaceutical interventions such as social distancing and lockdown to slow down the spread of the pandemic.,0.362183,A hybrid multi-scale model of COVID-19 transmission dynamics to assess the potential of non-pharmaceutical interventions,Not Available
3,0uzma5vr,npis are considered to be critical in order to at least delay the peak number of infected individuals and to prevent the health care system becoming overwhelmed by the number of patients to treat in hospitals or in intensive care units (icus).,0.360318,Estimate of the development of the epidemic reproduction number Rt from Coronavirus SARS-CoV-2 case data and implications for political measures based on prognostics,Not Available
4,cnmijcwv,"the model found that desire, perceived behavioral control, frequency of past behavior, and non-pharmaceutical interventions predicted tourists ' intention but perceptions of 2009 h1n1 had nil effect on desire and intention .",0.308177,The impact of non-pharmaceutical interventions for 2009 H1N1 influenza on travel intentions: A model of goal-directed behavior,Not Available
5,jtwb17u8,"historical and near-real time human movement data, obtained from baidu location-based service, were used to derive the intensity of travel restrictions and contact reductions across china.",0.204339,Effect of non-pharmaceutical interventions for containing the COVID-19 outbreak in China,Not Available
6,velir6gb,", using a thermometer, wearing a face mask) and inadequate hand washing and household disinfection practices .",0.171863,Pandemic preparedness: perceptions of vulnerable migrants in Thailand towards WHO-recommended non-pharmaceutical interventions: a cross-sectional study,Not Available


Define All The Questions

In [27]:
all_topics=[
    'What is known about transmission, incubation, and environmental stability?',
    'What do we know about COVID-19 risk factors?',
    'What do we know about virus genetics, origin, and evolution?',
    'What do we know about vaccines and therapeutics?',
    'What do we know about non-pharmaceutical interventions?',
    'What has been published about medical care?',
    'What do we know about diagnostics and surveillance?'
    'What has been published about information sharing and inter-sectoral collaboration?',
    'What has been published about ethical and social science considerations?'
]
topic_area = {}

#0
#What is known about transmission, incubation, and environmental stability?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water")
kw_list.append("2019-nCoV,SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, person to person, human to human, interpersonal contact, air, water,fecal, surfaces, aerisol, transmission, shedding")
pm_kw_list.append("2019-nCoV, transmission, shedding")

question_list.append( "How long is the incubation period for the virus")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, hours, days, period")
pm_kw_list.append("2019-nCoV, incubation period")

question_list.append("Can the virus be transmitted asymptomatically or during the incubation period")
kw_list.append("2019-nCoV, COVID-19, coronavirus, novel coronavirus, asymptomatic, person to person, human to human, transmission")
pm_kw_list.append("2019-nCoV, asymptomatic, transmission")

question_list.append("What is the quantity of asymptomatic shedding")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, asymptomatic, shedding, percentage, rate, quantity, pediatric")
pm_kw_list.append("2019-nCoV, asymptomatic, shedding")

question_list.append("How does temperature and humidity affect the tramsmission of 2019-nCoV")
kw_list.append("2019-nCoV, COVID-19, coronavirus, novel coronavirus, temperature, humidity")
pm_kw_list.append("2019-nCoV, temperature, humidity")

question_list.append("How long can 2019-nCoV remain viable on inanimate, environmental, or common surfaces")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, inanimate, environmental, touch, copper, plastic, steel, wood, fabric, glass, porous, nonporous")
pm_kw_list.append("2019-nCoV, surface")

question_list.append("What types of inanimate or environmental surfaces affect transmission, survival, or  inactivation of 2019-nCov")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, inanimate, environmental, touch, copper, plastic, steel, wood, fabric, glass, porous, nonporous")
pm_kw_list.append("2019-nCoV, surface")

question_list.append("Can the virus be found in nasal discharge, sputum, urine, fecal matter, or blood")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, shedding, body fluid")
pm_kw_list.append("2019-nCoV, body fluids")

topic_area['What is known about transmission, incubation, and environmental stability?'] = list(zip(question_list,kw_list, pm_kw_list))



#1
#What do we know about COVID-19 risk factors?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What risk factors contribute to the severity of 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, susceptible, smoking, smoker, neonates, pregnant, socio-economic, behavioral, age, elderly, young, old, children")
pm_kw_list.append("2019-nCoV, risk factors")

question_list.append("How does hypertension affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, hypertension, blood pressure, comorbidity")
pm_kw_list.append("2019-nCoV, hypertension, comorbidity")

question_list.append("How does heart disease affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, heart disease, comorbidity")
pm_kw_list.append("2019-nCoV, heart disease, comorbidity")

question_list.append("How does copd affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, copd, chronic obstructive pulmonary disease")
pm_kw_list.append("2019-nCoV, copd, chronic obstructive pulmonary disease")

question_list.append("How does smoking affect 2019-nCoV patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, smoking, smoker")
pm_kw_list.append("2019-nCoV, smoking, smoker")

question_list.append("How does pregnancy affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, pregnant, pregnancy")
pm_kw_list.append("2019-nCoV, pregnant, pregnancy")

question_list.append("What are the case fatality rates for 2019-nCoV patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, fatality rate")
pm_kw_list.append("2019-nCoV, fatality, statistics, death")

question_list.append("What is the case fatality rate in Italy")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, fatality rate, Italy")
pm_kw_list.append("2019-nCoV, fatality, statistics, death")

question_list.append("What public health policies prevent or control the spread of 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, guidance, prevention measures, public health, community, prevention, administration, government, health department, policy, control measures, travel")
pm_kw_list.append("2019-nCoV, guidance, public health,  policy, control measures")

topic_area['What do we know about COVID-19 risk factors?'] = list(zip(question_list,kw_list, pm_kw_list))


#2
#What do we know about virus genetics, origin, and evolution?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("Can animals transmit 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, animals, zoonotic, farm, spillover, animal to human, human to animal, reservoir")
pm_kw_list.append("2019-nCoV, spillover, reservoir")

question_list.append("What animal did 2019-nCoV come from")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, animals, zoonotic, farm, spillover, animal to human, bats, snakes, exotic animals")
pm_kw_list.append("2019-nCoV, zoonotic")

question_list.append("What real-time genomic tracking tools exist")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, real-time, gene, typing, tracking, software, reporting")
pm_kw_list.append('"2019-nCoV, real-time, genomic, tracking')

question_list.append("What regional genetic variations (mutations) exist")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, geography, region, genome, mutations")
pm_kw_list.append("2019-nCoV, geneome, region")

question_list.append("What effors are being done in asia to prevent further outbreaks")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, surveylance, wildlife, livestock, monitoring, asia, prevent, prevention, outbreaks")
pm_kw_list.append("2019-nCoV, surveylance")

question_list.append("What is the weighted prevalence of sars-cov-2 or covid-19 in general population")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("antibody, prevalence, COVID-19")

topic_area['What do we know about virus genetics, origin, and evolution?'] = list(zip(question_list,kw_list, pm_kw_list))

#3
#What do we know about vaccines and therapeutics?
question_list = []
kw_list = []
pm_kw_list = []

question_list.append("Does the presense neutralizing antibodies correlate to disease protection")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("sars-cov-2, neutralizing antibody")

question_list.append("What drugs or therapies are being investigated")
kw_list.append("2019-nCoV,  COVID-19, coronavirus, novel coronavirus, drug, antiviral, testing, clinical trial, study")
pm_kw_list.append("2019-nCoV,  drug, therapy")

question_list.append("What clinical trials for hydroxychloroquine have been completed")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, hydroxychloroquine, clinical trial")
pm_kw_list.append("2019-nCoV, hydroxychloroquine")

question_list.append("What antiviral drug clinical trials have been completed")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, clinical trial")
pm_kw_list.append("2019-nCoV, antiviral")

question_list.append("Are anti-inflammatory drugs recommended")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, ibuprofen, advil, NSAID, anti-inflamatory, treatment")
pm_kw_list.append('2019-nCoV, ibuprofen, NSAID')

topic_area['What do we know about vaccines and therapeutics?'] = list(zip(question_list,kw_list, pm_kw_list))


#4
#What do we know about non-pharmaceutical interventions?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("Which non-pharmaceutical interventions limit tramsission")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, non-pharmaceutical interventions, npi")
pm_kw_list.append("2019-nCoV, npi")

question_list.append("What are most important barriers to compliance")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, non-pharmaceutical interventions, npi")
pm_kw_list.append('2019-nCoV, npi, barrier to compliance')

topic_area['What do we know about non-pharmaceutical interventions?'] = list(zip(question_list,kw_list, pm_kw_list))

#5
#What has been published about medical care?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("How does extracorporeal membrane oxygenation affect 2019-nCoV patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, extracorporeal membrane oxygenation, ecmo")
pm_kw_list.append('2019-nCoV, ecmo')

question_list.append("What telemedicine and cybercare methods are most effective")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, telemedicine, 5G, cell phone, cyber, cybercare, information technolog, remote, over the phone, internet, web")
pm_kw_list.append('2019-nCoV, telemedicine, cybercare')

question_list.append("How is artificial intelligence being used in real time health delivery")
kw_list.append("2019-nCoV, ai, real-time")
pm_kw_list.append('')

question_list.append("What adjunctive or supportive methods can help patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, adjunctive, supportive")
pm_kw_list.append('')

topic_area['What has been published about medical care?'] = list(zip(question_list,kw_list, pm_kw_list))

#6
#What do we know about diagnostics and surveillance?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, diagnosis, tools, detetion")
pm_kw_list.append('2019-nCoV, diagnostic, tools, detetion')

question_list.append("What is being done to increase testing capacity or throughput")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, diagnostic, testing, throughput")
pm_kw_list.append("2019-nCoV, testing capacity OR throughput")

question_list.append("What point of care tests are exist or are being developed")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("2019-nCoV, point-of-care")

question_list.append("What is the minimum viral load for detection")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("2019-nCoV, viral load")

question_list.append("What markers are used to detect or track COVID-19")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("2019-nCoV, markers")

topic_area['What do we know about diagnostics and surveillance?'] = list(zip(question_list,kw_list, pm_kw_list))



#7
#What has been published about information sharing and inter-sectoral collaboration?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append('What collaborations are happening within the research community')
kw_list.append('inter-sectorial, international, collaboration, global, coronavirus, novel coronavirus, sharing')
pm_kw_list.append('2019-nCoV, collaboration, sharing')

question_list.append("What communication strategies are effective for high risk populations")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("2019-nCoV, communication, at risk")

topic_area['What has been published about information sharing and inter-sectoral collaboration?'] = list(zip(question_list,kw_list, pm_kw_list))


#8
#What has been published about ethical and social science considerations?
question_list = []
kw_list = []
pm_kw_list = []

question_list.append("What are the major ethical issues related pandemic outbreaks")
kw_list.append("ehtics, pandemic")
pm_kw_list.append("ethics, pandemic")

question_list.append("How do pandemics affect the physical and/or psychological health of doctors and nurses")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, caregivers, health care workers")
pm_kw_list.append("2019-nCoV, physical OR psychological")

question_list.append("What strategies can help doctors and nurses cope with stress in a pandemic")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, caregivers, health care workers")
pm_kw_list.append("2019-nCoV, physical OR psychological")

question_list.append("What factors contribute to rumors and misinformation")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, social media")
pm_kw_list.append("2019-nCoV, misinformation OR social media")

topic_area['What has been published about ethical and social science considerations?'] = list(zip(question_list,kw_list, pm_kw_list))



#-1
# Other interesting Questions
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What is the immune system response to 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, immune, immune system, response, immunity, antibodies")
pm_kw_list.append('2019-nCoV, immune system, immunity, antibodie')

question_list.append("Can personal protective equipment prevent the transmission of 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, ppe, masks, gloves, face shields, gown, eye protection")
pm_kw_list.append('2019-nCoV, ppe')

question_list.append("Can 2019-nCoV infect patients a second time")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, reinfected, multiple infections, second time, permenant immunity")
pm_kw_list.append('2019-nCoV, reinfected')

topic_area['Other interesting Questions'] = list(zip(question_list,kw_list, pm_kw_list))


def runAllQuestionsByTopic(topic_dict, topic_name):
    for q,kw, pm_kw in topic_dict[topic_name]:
        if q:
            searchDatabase(q, kw, pysearch, luceneDir, pm_kw=pm_kw, minDate=minDate)

In [28]:
question_list = []
kw_list = []
pm_kw_list=[]

question_list.append("What is the weighted prevalence of sars-cov-2 or covid-19 in general population")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("antibody, prevalence, COVID-19")


searchDatabase(question_list[0], kw_list[0], pysearch, luceneDir, pm_kw=pm_kw_list[0], minDate='2020/01/01', k=20)

100%|██████████| 20/20 [00:01<00:00, 18.00it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,pth2d40p,"adults represent the population with the highest infection rate ; however, neonates, (p.-r.",0.517242,"Asymptomatic carrier state, acute respiratory disease, and pneumonia due to severe acute respiratory syndrome coronavirus 2 (SARSCoV-2): Facts and myths",Not Available
1,ejg393zs,"the case fatality rate in patients infected with sarc-cov-2 is 4. 3 % , and the results indicate that the mortality is higher in elderly individuals and patients with chronic conditions including patients with coronary artery disease, diabetes, chronic pulmonary disease, and hypertension.",0.458472,Novel Coronavirus Disease 2019 (COVID-19): An Emerging Infectious Disease in the 21st Century,Not Available
2,6vt60348,"the true number of people currently infected with the virus, divided by the total population size .",0.456849,Estimation of SARS-CoV-2 Infection Prevalence in Santa Clara County,Not Available
3,2ftw85xw,"this commentary discusses the reasons for the fast spread of sars-cov-2 in three aspects : the infectious sources, including the biological nature of the virus ; the susceptible population ; and the transmission routes.",0.360217,Facing the COVID-19 outbreak: What should we know and what could we do?,Not Available
4,ne0qwv56,"background 33 an outbreak of a novel coronavirus (sars-cov-2)-infected pneumonia was first occurred in wuhan, china, in december 2019 and then spread rapidly to 35 other regions. it has been declared that at least one confirmed case infected by 36 sars-cov-2 was found in each province of china by late january 2020. 37 we collected the time series data of the cumulative number of confirmed infected, 39 dead, and cured cases from the health commissions in 31 provinces in mainland china. 40 a simple descriptive model in a logistic form was formulated to infer the intrinsic 41 epidemic rules of covid-19. furthermore, we compared the intrinsic epidemic rules 42 of covid-19 in hubei with that of severe acute respiratory syndrome (sars) in 43 beijing, which was obtained from the ministry of public health of china in 2003. 44 the feature of data of three population groups in all the provinces could be well 46 captured (r 2 > 0. 95 ) by the proposed descriptive model with slight variation among 47 (which was not peer-reviewed) the copyright holder for this preprint.",0.323962,"Intrinsic growth rules of patients infected, dead and cured with 2019 novel coronavirus in mainland China",Not Available
5,s7uqawbd,"by press time, more than 70,000 infected people had been confirmed worldwide .",0.255771,Rapid colorimetric detection of COVID-19 coronavirus using a reverse tran-scriptional loop-mediated isothermal amplification (RT-LAMP) diagnostic plat-form: iLACO,Not Available
6,rmia5w3x,a word count for text : 2460 words all rights reserved.,0.048428,Clinical and pathological characteristics of 2019 novel coronavirus disease (COVID-19): a systematic review,Not Available


In [29]:
minDate='2020/01/01'

What is known about transmission, incubation, and environmental stability?

In [30]:
runAllQuestionsByTopic(topic_area, 'What is known about transmission, incubation, and environmental stability?')

100%|██████████| 10/10 [00:00<00:00, 20.37it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,mz06mkqf,persistent shedding of sars-cov-2 in stools of infected children indicates the potential for the virus to be transmitted through fecal excretion .,0.718391,Prolonged presence of SARS-CoV-2 in feces of pediatric patients during the convalescent phase,Not Available
1,z4a8ru9z,"the virus is transmitted through droplets, close contact, and other means , and patients in the incubation period could potentially transmit the virus to other persons.",0.699239,2019-nCoV: new challenges from coronavirus,Not Available
2,xs7vkm19,"disease spread through both direct (droplet and person-to-person) as well as indirect contact (contaminated objects and airborne transmission) are indicated , supporting the use of airborne isolation precautions.",0.625732,Transmission Potential of SARS-CoV-2 in Viral Shedding Observed at the University of Nebraska Medical Center,Not Available
3,bzwxt9bq,"the new coronavirus was reported to spread via droplets, contact and natural aerosols from human-to-human .",0.381108,Expert Recommendations for Tracheal Intubation in Critically ill Patients with Noval Coronavirus Disease 2019,Not Available
4,ztcyvsoi,there is a new public health crises threatening the world with the emergence and spread of 2019 novel coronavirus (2019-ncov),0.300963,A Review of Coronavirus Disease-2019 (COVID-19),Not Available


100%|██████████| 10/10 [00:00<00:00, 13.87it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,8n2s0bl1,"in the healthcare environment, microorganisms ' cross-transmission between inanimate surfaces and patients or healthcare workers can lead to healthcare-associated infections.",0.506739,Copper Alloy Touch Surfaces in Healthcare Facilities: An Effective Solution to Prevent Bacterial Spreading,Not Available
1,tjplc5j6,"assessment of the risks posed by severe acute respiratory syndrome (sars) coronavirus (sars-cov) on surfaces requires data on survival of this virus on environmental surfaces and on how survival is affected by environmental variables, such as air temperature (at) and relative humidity (rh).",0.478299,Effects of Air Temperature and Relative Humidity on Coronavirus Survival on Surfaces,Not Available
2,k9xhphpl,"we therefore reviewed the literature on all available information about the persistence of human and veterinary coronaviruses on inanimate surfaces as well as inactivation strategies with biocidal agents used for chemical disinfection, e. g. in healthcare facilities. the analysis of 22 studies reveals that human coronaviruses such as severe acute respiratory syndrome (sars) coronavirus, middle east respiratory syndrome (mers) coronavirus or endemic human coronaviruses (hcov) can persist on inanimate surfaces like metal, glass or plastic for up to 9 days, but can be efficiently inactivated by surface disinfection procedures with 62e71 % ethanol, 0.",0.464713,Persistence of coronaviruses on inanimate surfaces and their inactivation with biocidal agents,Not Available
3,awp1nck0,"the previous study indicated that different human coronaviruses, such as sars ‐ cov and mers ‐ cov, can maintain infectious for a different time on inanimate surfaces.",0.434553,Uncertainties about the transmission routes of 2019 novel coronavirus,Not Available
4,f5xp2ebn,"the chemical disinfection of virus- contaminated non-porous inanimate surfaces was investigated using coxsackievirus b3, adenovirus type 5, parainfluenza virus type 3 and coronavirus 229e as representatives of important nosocomial viral pathogens.",0.401105,Chemical disinfection of non-porous inanimate surfaces experimentally contaminated with four human pathogenic viruses.,Not Available


100%|██████████| 10/10 [00:00<00:00, 12.63it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,33pxylb3,"mers-cov has been detected in the nasal and rectal secretions of infected camels , and mers-cov of this origin has full capacity to infect human airway epithelium in both in vitro and in vivo models.",0.620059,Middle East Respiratory Syndrome Coronavirus and the One Health concept,Not Available
1,ouca1bol,"clinical samples from all patients including nasopharyngeal swab (nps) / sputum, urine, and feces were collected for serial virus rna testing by standard qrt-pcr assay.",0.587198,Evaluation of SARS-CoV-2 RNA shedding in clinical specimens and clinical characteristics of 10 patients with COVID-19 in Macau,Not Available
2,gbkgzvc8,"graphical abstract highlights d sars-cov-2-infected ferrets exhibit elevated body temperature and virus replication d sars-cov-2 is shed in nasal washes, saliva, urine and feces d sars-cov-2 is effectively transmitted to naive ferrets by direct contact d sars-cov-2 infection leads acute bronchiolitis in infected ferrets",0.556308,Infection and Rapid Transmission of SARS-CoV-2 in Ferrets,Not Available


What do we know about COVID-19 risk factors?

In [31]:
runAllQuestionsByTopic(topic_area, 'What do we know about COVID-19 risk factors?')

100%|██████████| 10/10 [00:00<00:00, 17.02it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,7jjpnfay,"almost 70 % of patients susceptible to 2019-ncov are over age of 50 years, with extremely large proportion of critical illness and death of the elderly patients .",0.507629,Consensus on standardized diagnosis and treatment for osteoporotic vertebral compression fracture patients during epidemic of corona virus disease 2019,Not Available
1,uqkcbs2s,"in current severe global emergency situation of 2019-ncov outbreak, it is imperative to identify vulnerable and susceptible groups for effective protection and care .",0.494377,"Bulk and single-cell transcriptomics identify tobacco-use disparity in lung gene expression of ACE2, the receptor of 2019-nCov",Not Available
2,cdjhmk5z,"neonates are thought to be susceptible to the virus because their immune system is not well developed , which is of great concern to neonatal medical service providers.",0.456040,Managing neonates with respiratory failure due to SARS-CoV-2 – Authors' reply,Not Available
3,2mohptl2,"secondary endpoints included the clinical severity of covid-19, neonatal mortality, admission to neonatal intensive care unit (nicu), and the incidence of acute respiratory distress syndrome (ards) of pregnant women and newborns .",0.388002,Clinical features and the maternal and neonatal outcomes of pregnant women with coronavirus disease 2019,Not Available
4,r5jz7966,no one has progressed to critical pneumonia .,0.358068,Analysis of the pregnancy outcomes in pregnant women with COVID-19 in Hubei Province,Not Available


100%|██████████| 10/10 [00:00<00:00, 10.14it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,u1i5onqr,"calculating the total case fatality rate (cfr) of italy (2020-03-04), about 7. 9 % of confirmed cases passed away .",0.807883,Understanding Epidemic Data and Statistics: A case study of COVID-19,Not Available
1,wmxemk5r,comparison between case fatality rates in china and italy are identical at 2. 3 .,0.768221,Similarity in Case Fatality Rates (CFR) of COVID-19/SARS-COV-2 in Italy and China,Not Available
2,bxwcixd7,"92 %, with a high covid-19 fatality rate in italy (7. 7 % ), iran (5.",0.746093,"Global, Regional and National Incidence and Case-fatality rates of Novel Coronavirus (COVID-19) across 154 countries and territories: A systematic assessment of cases reported from January to March 16, 2020",Not Available
3,lgqseo8t,"these estimates differ markedly from those documented in countries such as spain and italy, in which covid-19 case fatality rates exceed 8 % (case of italy) and from the percentage of patients who have required intensive care, which has ranged from 9 % to 11 % of patients in mediterranean european countries.",0.668458,"COVID-19 in Colombia endpoints. Are we different, like Europe?",Not Available
4,ejg393zs,"the case fatality rate in patients infected with sarc-cov-2 is 4. 3 % , and the results indicate that the mortality is higher in elderly individuals and patients with chronic conditions including patients with coronary artery disease, diabetes, chronic pulmonary disease, and hypertension.",0.540315,Novel Coronavirus Disease 2019 (COVID-19): An Emerging Infectious Disease in the 21st Century,Not Available


100%|██████████| 10/10 [00:01<00:00,  8.85it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,q856rx6b,these findings can help the government and health departments better communicate with the public on health and translate public health needs into practice to create targeted measures to prevent and control the spread of covid-19 .,0.687958,Chinese Public Attention to COVID-19 Epidemic: Based on Social Media,Not Available
1,zppc6p20,global health authorities should take immediate measures to prevent the outbreaks of such emerging and reemerging pathogens across the globe to minimize the disease burden locally and globally .,0.612341,"Novel coronavirus 2019-nCoV: prevalence, biological and clinical characteristics comparison with SARS-CoV and MERS-CoV",Not Available
2,l4o7nicc,"although these measures might not prevent widespread transmission of the virus in the united states, they are being implemented to 1 ) slow the spread of illness ; 2) provide time to better prepare state and local health departments, health care systems, businesses, educational organizations, and the general public in the event that widespread transmission occurs ; and 3) better characterize covid-19 to guide public health recommendations and the development and deployment of medical countermeasures, including diagnostics, therapeutics, and vaccines.",0.573463,"Update: Public Health Response to the Coronavirus Disease 2019 Outbreak - United States, February 24, 2020",Not Available
3,ejg393zs,"currently, due to the lack of an effective treatment and vaccine, the best way to deal with the covid-19 disease is to prevent transmission and spread of the virus and to execute personal protective measures .",0.569224,Novel Coronavirus Disease 2019 (COVID-19): An Emerging Infectious Disease in the 21st Century,Not Available
4,9sdehh8v,"in a meeting on january 30,2020, per the international health regulations (ihr, 2005), the outbreak was declared by the who a public health emergency of international concern (pheic) as it had spread to 18 countries with four countries reporting human-to-human transmission.",0.559437,"Features, Evaluation and Treatment Coronavirus (COVID-19)",Not Available
5,s9j4yv3j,"this threat is posing a severe challenge on the diagnosis and treatment of 2019-ncov infection, the prevention and control of hospital cross infection of medical staff .",0.532043,Advices on the prevention and control of nosocomial infection of novel coronavirus within children’s hospitals,Not Available
6,ikxbwbqf,"following an outbreak of pneumonia caused by 2019 novel coronavirus (2019-ncov) which has spread in china, the national health commission of china issued public health policies and implemented interventions for control and prevention of the epidemic.",0.498386,Proposed prevention and control of 2019 novel coronavirus infection in neonates,Not Available
7,xk5igdwo,"what you need to knowconsider covid-19 infection in anyone with cough, fever, or breathlessness who has had contact with someone with covid-19, or has returned from a high risk area in the 14 days before the onset of symptomsevery effort should be made to avoid in-person assessment of patients with possible covid-19 in primary caregp surgeries should plan ahead and develop clear protocols for managing possible cases, including isolation procedures, personal protective equipment, seeking specialist advice, and decontaminationif covid-19 infection is suspected in someone attending the practice, isolate the patient in a room (away from other patients and staff), close the door, and ask the patient to call nhs 111the guidance may change so it is essential to look at the latest guidance online (box 1) the uk recorded its first confirmed case of acute respiratory infection due to coronavirus disease 2019 (covid-19) on 31 january 2020 and responded by quarantining at-risk individuals to contain the spread of infection.",0.408110,Coronavirus disease 2019 (covid-19): a guide for UK GPs,Not Available


What do we know about virus genetics, origin, and evolution?


In [32]:
runAllQuestionsByTopic(topic_area, 'What do we know about virus genetics, origin, and evolution?')

100%|██████████| 10/10 [00:00<00:00, 13.41it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,5f42du0b,"although bats are likely reservoir hosts for 2019-ncov , the identity of 31 any intermediate host facilitating transfer to humans is unknown.",0.471380,Identification of 2019-nCoV related coronaviruses in Malayan pangolins in southern China,Not Available
1,1qkwsh6a,"coronaviruses are the well-known cause of severe respiratory, enteric and systemic infections in a wide range of hosts including man, mammals, fish, and avian.",0.415257,"Emerging novel coronavirus (2019-nCoV)—current scenario, evolutionary perspective based on genome analysis and recent developments",Not Available
2,rxrlbw60,"with a seemingly comparable chain of events as the origin of sars-cov, the initial infections with 2019-ncov appears to be linked to contact with animals in wet markets. even though human-to-human ralph et al.-2019-ncov wuhan and human-to-human transmission j infect dev ctries 2020 ; 14 (1) : 3-17.",0.371222,"2019-nCoV (Wuhan virus), a novel Coronavirus: Human-to-human transmission, travel-related cases, and vaccine readiness",Not Available
3,2inlyd0t,"the outbreak of 2019-ncov pneumonia in the city of wuhan, china has resulted in more than 70,000 laboratory confirmed cases, and recent studies showed that 2019-ncov (sars-cov-2) could be of bat origin but involve other potential intermediate hosts .",0.369339,Are pangolins the intermediate host of the 2019 novel coronavirus (2019-nCoV) ?,Not Available
4,pwvcwlh8,"taken together, our results suggest that homologous recombination within the spike glycoprotein may contribute to cross-species transmission from snake to humans. this article is protected by copyright.",0.352358,Cross‐species transmission of the newly identified coronavirus 2019‐nCoV,Not Available
5,vvhhpchm,the present outbreak of a coronavirus-associated acute respiratory disease called coronavirus disease 19 (covid-19) is the third documented spillover of an animal coronavirus to humans in only two decades that has resulted in a major epidemic.,0.350618,The species Severe acute respiratory syndrome-related coronavirus: classifying 2019-nCoV and naming it SARS-CoV-2,Not Available
6,awitk3se,"although several animals have been speculated to be a reservoir for covid-19, no animal reservoir has been already confirmed .",0.347768,COVID-19 (Novel Coronavirus 2019) - recent trends,Not Available
7,z9dolxky,"to support the hypothesis of zoonotic origin of 2019-ncov stemming from the huanan seafood market, the index case should have had exposure history related to the market and the virus should have been identified from animals sold at the market .",0.293274,"Initial Cluster of Novel Coronavirus (2019-nCoV) Infections in Wuhan, China Is Consistent with Substantial Human-to-Human Transmission",Not Available


100%|██████████| 10/10 [00:00<00:00, 13.38it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,b5329o75,"ironically, even after a decade of research on coronavirus, still there are no licensed vaccines or therapeutic agents to treat coronavirus infection which highlights an urgent need to develop effective vaccines or post-exposure prophylaxis to prevent future epidemics.",0.627747,Perspectives on monoclonal antibody therapy as potential therapeutic intervention for Coronavirus disease-19 (COVID-19),Not Available
1,5fg87lvu,"at this stage, preparedness, transparency, and sharing of information are crucial to risk assessments and beginning outbreak control activities .",0.604846,The SARS-CoV-2 outbreak: what we know,Not Available
2,h5sox8bq,"to prevent the next epidemic and pandemic related to these interfaces, we call for research and investment in three areas : 1) surveillance among wildlife to identify the high-risk pathogens they carry ; 2) surveillance among people who have contact with wildlife to identify early spillover events ; and 3) improvement of market biosecurity regarding the wildlife trade .",0.598839,A strategy to prevent future pandemics similar to the 2019-nCoV outbreak,Not Available
3,rql9fjug,"as the outbreak of covid-19 causes by the severe acute respiratory syndrome coronavirus 2 (sars-cov-2) progresses within china and beyond, rapidly available epidemiological data are needed to guide strategies for situational awareness and intervention .",0.577554,Analyzing the Epidemiological Outbreak of COVID-19: A Visual Exploratory Data Analysis (EDA) Approach,Not Available
4,ejg393zs,"moreover, websites such as the world health organization (who) and the centers for disease control and prevention (cdc) were searched to retrieve updated data and statistics regarding the novel coronavirus. we extracted data on the epidemiology, pathogenesis, virology, clinical manifestations, transmission routes, diagnosis, treatment, and prevention measures. results : from the 1416 articles identified in the initial search, 53 were remained after title and abstract screening. after full-text review, 37 articles were eligible to include in our study. incubation period for covid-19 is between 2-10 days, according to the world health organization (who). the case fatality rate in patients infected with sarc-cov-2 is 4. 3 %, and the results indicate that the mortality is higher in elderly individuals and patients with chronic conditions including patients with coronary artery disease, diabetes, chronic pulmonary disease, and hypertension. the mortality rate in healthy subjects is less than 1 %. conclusion : the outbreak caused by the novel coronavirus is larger than the previous human coronaviruses, showing that the sars-cov-2 is an extremely contagious virus. however, the mortality rate of covid-19 is lower than that of other coronaviruses diseases such as sars or mers and other viruses like hiv and ebola. currently, due to the lack of an effective treatment and vaccine, the best way to deal with the covid-19 disease is to prevent transmission and spread of the virus and to execute personal protective measures .",0.481911,Novel Coronavirus Disease 2019 (COVID-19): An Emerging Infectious Disease in the 21st Century,Not Available


100%|██████████| 10/10 [00:00<00:00, 24.79it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,pth2d40p,"adults represent the population with the highest infection rate ; however, neonates, (p.-r.",0.517242,"Asymptomatic carrier state, acute respiratory disease, and pneumonia due to severe acute respiratory syndrome coronavirus 2 (SARSCoV-2): Facts and myths",Not Available
1,rmia5w3x,a word count for text : 2460 words all rights reserved.,0.048428,Clinical and pathological characteristics of 2019 novel coronavirus disease (COVID-19): a systematic review,Not Available


What do we know about non-pharmaceutical interventions?

In [33]:
runAllQuestionsByTopic(topic_area, 'What do we know about non-pharmaceutical interventions?')

100%|██████████| 10/10 [00:01<00:00,  9.28it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,kqntu6ej,"in impact of non-pharmaceutical interventions (npis) to reduce covid-19 mortality and healthcare demand , imperial college covid-19 response team, london, 2020.",0.514955,"Commentary on Ferguson, et al., “Impact of Non-pharmaceutical Interventions (NPIs) to Reduce COVID-19 Mortality and Healthcare Demand”",Not Available
1,vy1mu04l,"much of the uncertainty about the progression of the covid-19 pandemic stems from questions about when and how non-pharmaceutical interventions (npi) by governments, in particular social distancing measures , are implemented, to what extent the population complies with these measures, and how compliance changes through time.",0.436335,How will this continue? Modelling interactions between the COVID-19 pandemic and policy responses,Not Available
2,swcq2wfl,public health resorted to non-pharmaceutical interventions such as social distancing and lockdown to slow down the spread of the pandemic.,0.362183,A hybrid multi-scale model of COVID-19 transmission dynamics to assess the potential of non-pharmaceutical interventions,Not Available
3,0uzma5vr,npis are considered to be critical in order to at least delay the peak number of infected individuals and to prevent the health care system becoming overwhelmed by the number of patients to treat in hospitals or in intensive care units (icus).,0.360318,Estimate of the development of the epidemic reproduction number Rt from Coronavirus SARS-CoV-2 case data and implications for political measures based on prognostics,Not Available
4,cnmijcwv,"the model found that desire, perceived behavioral control, frequency of past behavior, and non-pharmaceutical interventions predicted tourists ' intention but perceptions of 2009 h1n1 had nil effect on desire and intention .",0.308177,The impact of non-pharmaceutical interventions for 2009 H1N1 influenza on travel intentions: A model of goal-directed behavior,Not Available
5,jtwb17u8,"historical and near-real time human movement data, obtained from baidu location-based service, were used to derive the intensity of travel restrictions and contact reductions across china.",0.204339,Effect of non-pharmaceutical interventions for containing the COVID-19 outbreak in China,Not Available
6,velir6gb,", using a thermometer, wearing a face mask) and inadequate hand washing and household disinfection practices .",0.171863,Pandemic preparedness: perceptions of vulnerable migrants in Thailand towards WHO-recommended non-pharmaceutical interventions: a cross-sectional study,Not Available


100%|██████████| 10/10 [00:00<00:00, 14.47it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,we62087x,"effective public health interventions, including social distancing, contact tracing, and isolation / quarantine rely on the rapid and accurate identification of confirmed cases.",0.412371,"Sentinel Event Surveillance to Estimate Total SARS-CoV-2 Infections, United States",Not Available
1,kqntu6ej,"in impact of non-pharmaceutical interventions (npis) to reduce covid- 19 mortality and healthcare demand , imperial college covid-19 response team, london, 2020.",0.307130,"Commentary on Ferguson, et al., “Impact of Non-pharmaceutical Interventions (NPIs) to Reduce COVID-19 Mortality and Healthcare Demand”",Not Available
2,velir6gb,"other potential barriers to npi implementation include limited experience applying these interventions (e. g., using a thermometer, wearing a face mask) and inadequate hand washing and household disinfection practices .",0.302923,Pandemic preparedness: perceptions of vulnerable migrants in Thailand towards WHO-recommended non-pharmaceutical interventions: a cross-sectional study,Not Available
3,vy1mu04l,"much of the uncertainty about the progression of the covid-19 pandemic stems from questions about when and how non-pharmaceutical interventions (npi) by governments, in particular social distancing measures , are implemented, to what extent the population complies with these measures, and how compliance changes through time.",0.279524,How will this continue? Modelling interactions between the COVID-19 pandemic and policy responses,Not Available
4,jtwb17u8,"historical and near-real time human movement data, obtained from baidu location-based service, were used to derive the intensity of travel restrictions and contact reductions across china.",0.253653,Effect of non-pharmaceutical interventions for containing the COVID-19 outbreak in China,Not Available
5,pth2d40p,"adults represent the population with the highest infection rate ; however, neonates , (p.-r.",0.108962,"Asymptomatic carrier state, acute respiratory disease, and pneumonia due to severe acute respiratory syndrome coronavirus 2 (SARSCoV-2): Facts and myths",Not Available


What has been published about medical care?

In [34]:
runAllQuestionsByTopic(topic_area, 'What has been published about medical care?')

100%|██████████| 10/10 [00:00<00:00, 12.22it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,mqwgrwdb,"therefore, the number of patients with severe covid-19 requiring extracorporeal membranous oxygenation (ecmo) is expected to increase .",0.646871,Japan ECMOnet for COVID-19: telephone consultations for cases with severe respiratory failure caused by COVID-19,Not Available
1,dlwph5za,the objective of this study is to compare the outcomes of mers-cov patients before and after the availability of extracorporeal membrane oxygenation (ecmo) as a rescue therapy in severely hypoxemic patients who failed conventional strategies .,0.635612,Extracorporeal membrane oxygenation for severe Middle East respiratory syndrome coronavirus,Not Available
2,cmtsu73g,"although acute kidney injury (aki) is the most frequent complication in patients receiving extracorporeal membrane oxygenation (ecmo), few studies have been conducted on the risk factors of aki.",0.632820,Risk Factors for Acute Kidney Injury and In-Hospital Mortality in Patients Receiving Extracorporeal Membrane Oxygenation,Not Available
3,c0rbum7p,this report highlights the severity of late-onset complications of catheter placements and shows the potential of ecmo treatment for the management of acute illnesses with bridge to recovery.,0.549763,The answer is blowing in the wind: an uncommon cause for severe ARDS accompanied by circulatory insufficiency requiring extracorporeal membrane oxygenation,Not Available
4,6sscf1lq,"during his ecmo support stage, sudden occurred decreasing of ecmo flow implied increasing intrathoracic pressure .",0.521490,Extracorporeal membrane oxygenation with prone position ventilation successfully rescues infantile pertussis: a case report and literature review,Not Available
5,1e5wpt3x,"compared with patients with normal bnp, patients with high bnp were more likely to develop severe pneumonia, and receive tracheal cannula, invasive mechanical ventilation, continuous renal replacement therapy, extracorporeal membrane oxygenation, and be admitted to the intensive care unit .",0.516767,Clinical features and outcomes of 2019 novel coronavirus-infected patients with high plasma BNP levels,Not Available
6,bir92bja,32 patients survived (76 %) and have been discharged alive from hospital .,0.360799,Mobile ECMO team for inter-hospital transportation of patients with ARDS: a retrospective case series,Not Available


100%|██████████| 10/10 [00:00<00:00, 28.32it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,azrqz6hf,"from a systems-thinking perspective, and in today ' s world of seamless boundaries and global interconnectivity, ai offers enormous potential for public health practitioners and policy makers to revolutionize healthcare and population health through focussed, context-specific interventions that promote cost-savings on therapeutic care, expand access to health information and services, and enhance individual responsibility for their health and well-being .",0.619234,Artificial Intelligence Applications in Tracking Health Behaviors During Disease Epidemics,Not Available
1,chrj5h72,"cybercare employs advanced technology to deliver services efficiently across the distributed networkfor example, using telemedicine, wearable sensors and cell phones to link patients to specialists and upload their medical data in near-real time ; using information technology (it) to rapidly detect, track, and contain the spread of a global pandemic ; or using cell phones to manage medical care in a disaster situation.",0.571669,Cybercare 2.0: meeting the challenge of the global burden of disease in 2030,Not Available
2,3uk2zfpo,"two independent groups last week reported that they had used artificial intelligence in different ways to find possible treatments for the novel coronavirus , named 2019-ncov.",0.487519,Artificial intelligence finds drugs that could fight the new coronavirus,Not Available
3,e6q92shw,"besides, the system automatically highlighted all lesion regions for faster examination .",0.358340,AI-assisted CT imaging analysis for COVID-19 screening: Building and deploying a medical AI system in four weeks,Not Available


100%|██████████| 10/10 [00:01<00:00,  9.82it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,c3geck81,the supportive care and non-specific treatment to ameliorate the symptoms of the patient are the only options currently.,0.681950,Traditional Chinese Medicine in the Treatment of Patients Infected with 2019-New Coronavirus (SARS-CoV-2): A Review and Perspective,Not Available
1,cqlg8go7,"several adjunctive pharmacologic interventions have been studied for their immunomodulatory effects, including macrolides, corticosteroids, cyclooxygenase-2 inhibitors, sirolimus, statins, anti-influenza immune plasma, and vitamin c, but none is recommended at present in severe rvis. evidence-based supportive care is the mainstay for management of severe respiratory viral infection.",0.572880,Critical care management of adults with community-acquired severe respiratory viral infection,Not Available
2,juz9jnfk,"[UNK] remdesivir, chloroquine, tocilizumab, and convalescent plasma may be effective .",0.466459,Insight into 2019 novel coronavirus — an updated intrim review and lessons from SARS-CoV and MERS-CoV,Not Available
3,plede1kn,"refractory covid-19 patients were more likely to receive oxygen, mechanical ventilation, expectorant, and adjunctive treatment including corticosteroid, antiviral drugs and immune enhancer (p < 0.",0.421335,"Clinical characteristics of refractory COVID-19 pneumonia in Wuhan, China | Clinical Infectious Diseases | Oxford Academic",Not Available
4,5fg87lvu,"highlights [UNK] the latest summary of the covid-19 outbreak in china ; [UNK] there might be an oral-fecal transmission of the virus ; [UNK] aggregates and consolidates the epidemiology, clinical manifestations, diagnosis, treatments and preventions of this new type of coronavirus.",0.368658,The SARS-CoV-2 outbreak: what we know,Not Available


What do we know about diagnostics and surveillance?

In [35]:
runAllQuestionsByTopic(topic_area, 'What do we know about diagnostics and surveillance?')

100%|██████████| 10/10 [00:00<00:00, 12.90it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,8gncbgot,"however, serological assays as well as point-of-care testing kits have not been developed but are likely in the near future.",0.566219,"Potential Rapid Diagnostics, Vaccine and Therapeutics for 2019 Novel Coronavirus (2019-nCoV): A Systematic Review",Not Available
1,z4u017sa,"diagnostic test based on real-time reverse transcription polymerase chain reaction assay (qrt-pcr) was the main means of confirmation, and sample collection was mostly throat swabs, which was easy to miss the diagnosis.",0.556754,Comparison of throat swabs and sputum specimens for viral nucleic acid detection in 52 cases of novel coronavirus (SARS-Cov-2) infected pneumonia (COVID-19),Not Available
2,j6y806qu,here we use computational tools from structural biology and machine learning to identify 2019-ncov t-cell and b-cell epitopes based on viral protein antigen presentation and antibody binding properties.,0.539724,Potential T-cell and B-cell Epitopes of 2019-nCoV,Not Available
3,gxstlzuk,"the tool also allows tracking of new viral mutations as the outbreak expands globally, which may help to accelerate the development of novel diagnostics, drugs and vaccines .",0.518947,Genome Detective Coronavirus Typing Tool for rapid identification and characterization of novel coronavirus genomes,Not Available
4,8p1agcm2,"effective countermeasures against the recent emergence and rapid expansion of the 2019-novel coronavirus (2019-ncov) require the development of data and tools to understand and monitor viral spread and immune responses. however, little information about the targets of immune responses to 2019-ncov is available. we used the immune epitope database and analysis resource (iedb) resource to catalog available data related to other coronaviruses, including sars-cov, which has high sequence similarity to 2019-ncov, and is the best-characterized coronavirus in terms of epitope responses. we identified multiple specific regions in 2019-ncov that have high homology to sars virus. parallel bionformatic predictions identified a priori potential b and t cell epitopes for 2019-ncov. the independent identification of the same regions using two approaches reflects the high probability that these regions are targets for immune recognition of 2019-ncov. we identified potential targets for immune responses to 2019-ncov and provide essential information for understanding human immune responses to this virus and evaluation of diagnostic and vaccine candidates .",0.492796,Candidate targets for immune responses to 2019-Novel Coronavirus (nCoV): sequence homology- and bioinformatic-based predictions,Not Available
5,s7uqawbd,"the accuracy, simplicity and versatility of the new developed method suggests that ilaco assays can be conveniently applied with for 2019-ncov threat control, even in those cases where specialized molecular biology equipment is not available .",0.481791,Rapid colorimetric detection of COVID-19 coronavirus using a reverse tran-scriptional loop-mediated isothermal amplification (RT-LAMP) diagnostic plat-form: iLACO,Not Available


100%|██████████| 10/10 [00:00<00:00, 13.05it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,s7uqawbd,"reaction time varied from 15-40 minutes, depending on the loading of virus in the collected samples .",0.534625,Rapid colorimetric detection of COVID-19 coronavirus using a reverse tran-scriptional loop-mediated isothermal amplification (RT-LAMP) diagnostic plat-form: iLACO,Not Available
1,scc9wee0,the median viral load in posterior oropharyngeal saliva or other respiratory specimens at presentation was 5 · 2 log 10 copies per ml (iqr 4 · 1-7 · 0).,0.468597,Temporal profiles of viral load in posterior oropharyngeal saliva samples and serum antibody responses during infection by SARS-CoV-2: an observational cohort study,Not Available


100%|██████████| 10/10 [00:00<00:00, 19.23it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,s7uqawbd,here we developed an isothermal lamp based method-ilaco (isothermal lamp based method for covid-19) to amplify a fragment of the orf1ab gene using 6 primers .,0.249123,Rapid colorimetric detection of COVID-19 coronavirus using a reverse tran-scriptional loop-mediated isothermal amplification (RT-LAMP) diagnostic plat-form: iLACO,Not Available
1,rmia5w3x,a word count for text : 2460 words all rights reserved.,0.174361,Clinical and pathological characteristics of 2019 novel coronavirus disease (COVID-19): a systematic review,Not Available
2,4ihv80au,"coronavirus disease 2019 and severe acute respiratory syndrome (sars)-related coronaviruses (e. g., 2019-ncov and sars-cov) are phylogenetically distantly related, but both are capable of infecting human hosts via the same receptor, angiotensin-converting enzyme 2, and cause similar clinical and pathological features, suggesting their phenotypic convergence. yet, the molecular basis that underlies their phenotypic convergence remains unknown. here, we used a recently developed molecular phyloecological approach to examine the molecular basis leading to their phenotypic convergence. our genome-level analyses show that the spike protein, which is responsible for receptor binding, has undergone significant darwinian selection along the branches related to 2019-ncov and sars-cov. further examination shows an unusually high proportion of evolutionary convergent amino acid sites in the receptor binding domain (rbd) of the spike protein between covid-19 and sars-related cov clades, leading to the phylogenetic uniting of their rbd protein sequences.",0.144832,Strong evolutionary convergence of receptor-binding protein spike between COVID-19 and SARS-related coronaviruses,Not Available
3,awitk3se,"the world health organization (who) has issued a warning that, although the 2019 novel coronavirus (covid-19) from wuhan city (china), is not pandemic, it should be contained to prevent the global spread. the covid-19 virus was known earlier as 2019-ncov. as of 12 february 2020, who reported 45,171 cases and 1115 deaths related to covid-19. covid-19 is similar to severe acute respiratory syndrome coronavirus (sars-cov) virus in its pathogenicity, clinical spectrum, and epidemiology. comparison of the genome sequences of covid-19, sars-cov, and middle east respiratory syndrome coronavirus (mers-cov) showed that covid-19 has a better sequence identity with sars-cov compared to mers cov. however, the amino acid sequence of covid-19 differs from other coronaviruses specifically in the regions of 1ab polyprotein and surface glycoprotein or s-protein .",0.130012,COVID-19 (Novel Coronavirus 2019) - recent trends,Not Available


What has been published about ethical and social science considerations?

In [36]:
runAllQuestionsByTopic(topic_area, 'What has been published about ethical and social science considerations?')

100%|██████████| 10/10 [00:00<00:00, 11.85it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,mgqm9311,"existing pandemic ethics discussions have advocated to engage the public in scarcity dilemmas and attend the local contexts and cultural perspectives that shape responses to a global health threat. this public engagement study thus considers the role of community and culture in the ethical apportionment of scarce health resources , specifically ventilators, during an influenza pandemic.",0.734881,Influence of Community and Culture in the Ethical Allocation of Scarce Medical Resources in a Pandemic Situation: Deliberative Democracy Study,Not Available
1,5f46aull,"the unprecedented outbreak of ebola virus disease (evd) in west africa has raised several novel ethical issues for global outbreak preparedness. it has also illustrated that familiar ethical issues in infectious disease management endure despite considerable efforts to understand and mitigate such issues in the wake of past outbreaks. to improve future global outbreak preparedness and response, we must examine these shortcomings and reflect upon the current state of ethical preparedness. to this end, we focus our efforts in this article on the examination of one substantial area : ethical guidance in pandemic plans. we argue that, due in part to their focus on considerations arising specifically in relation to pandemics of influenza origin, pandemic plans and their existing ethical guidance are ill-equipped to anticipate and facilitate the navigation of unique ethical challenges that may arise in other infectious disease pandemics. we proceed by outlining three reasons why this is so, and situate our analysis in the context of the evd outbreak and the threat posed by drug-resistant tuberculosis : (1) different infectious diseases have distinct characteristics that challenge anticipated or existing modes of pandemic prevention, preparedness, response, and recovery , (2) clear, transparent, context-specific ethical reasoning and justification within current influenza pandemic plans are lacking, and (3) current plans neglect the context of how other significant pandemics may manifest.",0.710840,"Ethics for pandemics beyond influenza: Ebola, drug-resistant tuberculosis, and anticipating future ethical challenges in pandemic preparedness and response",Not Available
2,qvklgp5t,"this paper explains the ethical importance of infectious diseases, and reviews four major ethical issues associated with pandemic influenza : the obligation of individuals to avoid infecting others, healthcare workers ' ' duty to treat ', allocation of scarce resources, and coercive social distancing measures .",0.659091,Pandethics,Not Available
3,jjb4k1um,"also taken into consideration are such issues as the distribution of food and vaccines, quarantines, work stoppage, both physical and social infrastructure, the role of military and police forces, and the effect of a pandemic, isolation, and quarantine on various industries .",0.655954,Shaping Ethical Guidelines for an Influenza Pandemic,Not Available
4,58czem0j,"summary : an honest and critical examination of the role of hcps during communicable disease outbreaks is needed in order to provide guidelines regarding professional rights and responsibilities, as well as ethical duties and obligations .",0.618367,On pandemics and the duty to care: whose duty? who cares?,Not Available
5,nfp1t3i4,we reviewed the important ethical challenges presented by pregnant women and highlighted the considerations for all vulnerable groups when planning for a pandemic at both the local and the national level.,0.583201,Pandemic Influenza and Pregnancy: An Opportunity to Reassess Maternal Bioethics,Not Available
6,q63ce5pi,"methods : online pandemic plans from national, provincial and territorial government websites were reviewed to identify : plans for children and families, and psychosocial and ethical issues. a survey was administered to gather participants ' perspectives on the 

100%|██████████| 10/10 [00:00<00:00, 12.77it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,yj2jh694,"december 2019, there have been more than 76,000 cases of covid-19 in china, causing more than 3,000 medical staff infections. due to covid-19 spreads quickly, is highly contagious, and can be fatal in severe cases, and there are no specific medicines, it poses a huge threat to the life and health of nurses and has a large impact on their emotional responses and coping strategies .",0.550984,Emotional responses and coping strategies of nurses and nursing college students during COVID-19 outbreak,Not Available
1,ezaoi7ip,"this descriptive study, which was conducted from may to june 2003, intended to identify staff stress and coping strategies among a sars team of nursing staff during the outbreak.",0.549443,Facing SARS: psychological impacts on SARS team nurses and psychiatric services in a Taiwan general hospital,Not Available
2,i3r3ff3t,"the areas for coping strategies were clear directives / precautionary measures, ability to give feedback to / obtain support from management, support from supervisors / colleagues, support from the family, ability to talk to someone and religious convictions .",0.469607,Psychological impact of the 2003 severe acute respiratory syndrome outbreak on health care workers in a medium size regional general hospital in Singapore,Not Available
3,juz9jnfk,"[UNK] remdesivir, chloroquine, tocilizumab, and convalescent plasma may be effective .",0.321703,Insight into 2019 novel coronavirus — an updated intrim review and lessons from SARS-CoV and MERS-CoV,Not Available
4,o0mhzx3g,"plusieurs initiatives ont ete prises pour assurer la continuite des soins et contenir l ' epidemie : creation en psychiatrie d ' unite covid + co-supervisee par des medecins generalistes ou internistes, restriction des consultations aux cas severes et redeploiement des soins en teleconsultation, accompagnement de type case-management pour les sorties precoces ou page 3 of 21 j o u r n a l p r e-p r o o f 3 management and coping strategies , prevention of addictions, etc.",0.310483,Assurer les soins aux patients souffrant de troubles psychiques en France pendant l’épidémie à SARS-CoV-2,Not Available
5,znuqdzdp,"secondly, self-coping styles included psychological and life adjustment, altruistic acts, team support, and rational cognition .",0.194644,A Qualitative Study on the Psychological Experience of Caregivers of COVID-19 Patients,Not Available


100%|██████████| 10/10 [00:00<00:00, 19.84it/s]


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link,PDF Link
0,ghcbttba,"given that the covid-19 event is recent and its potential impact on longer term global movement of people and resources have yet to be seen, we explore the social aspects of this crisis and highlight the impact of misinformation circulating on social media .",0.342299,What does “crisis” education look like?,Not Available
1,kyri2hny,"sylvie briand, director of infectious hazards management at who ' s health emergencies programme and architect of who ' s strategy to counter the infodemic risk, told the lancet, “ we know that every outbreak will be accompanied by a kind of tsunami of information, but also within this information you always have misinformation, rumours, etc. we know that even in the middle ages there was this phenomenon ”.",0.213188,How to fight an infodemic,Not Available
